In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
  SELECT
    date_built, ROUND(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms == 4
  GROUP BY date_built
  ORDER BY date_built ASC
  """).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
  SELECT
    date_built, ROUND(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3
  GROUP BY date_built
  ORDER BY date_built ASC
  """).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
  SELECT
    date_built, ROUND(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_lot >= 2000
  GROUP BY date_built
  ORDER BY date_built ASC
  """).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|280447.23|
|      2011|281413.45|
|      2012|295858.68|
|      2013|295142.13|
|      2014|294195.13|
|      2015|291788.36|
|      2016| 287812.6|
|      2017|282026.59|
+----------+---------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
  SELECT
    price, view
    FROM home_sales
    WHERE price >= 350000
  ORDER BY price ASC
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------+----+
|  price|view|
+-------+----+
|1250986|  89|
|1251616|  84|
|1251818|  94|
|1252469|  80|
|1253509|  85|
|1253637|  86|
|1254734|  85|
|1255252|  90|
|1255305|  98|
|1256433|  95|
|1256833|  82|
|1257157|  85|
|1258719|  88|
|1259627|  88|
|1259743|  77|
|1259916|  91|
|1259958|  89|
|1260462|  82|
|1260923|  75|
|1261005|  77|
+-------+----+
only showing top 20 rows

--- 0.7159323692321777 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
  SELECT
    price, view
    FROM home_sales
    WHERE price >= 350000
  ORDER BY price ASC
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------+----+
|  price|view|
+-------+----+
|1250986|  89|
|1251616|  84|
|1251818|  94|
|1252469|  80|
|1253509|  85|
|1253637|  86|
|1254734|  85|
|1255252|  90|
|1255305|  98|
|1256433|  95|
|1256833|  82|
|1257157|  85|
|1258719|  88|
|1259627|  88|
|1259743|  77|
|1259916|  91|
|1259958|  89|
|1260462|  82|
|1260923|  75|
|1261005|  77|
+-------+----+
only showing top 20 rows

--- 0.2703876495361328 seconds ---


In [27]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
p_df = df.write.partitionBy("date_built").mode("overwrite").parquet("homesales_partition")

In [29]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet("homesales_partition")

In [30]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_homesales')

In [31]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
  SELECT
    price, view
    FROM home_sales
    WHERE price >= 350000
  ORDER BY price ASC
  """).show()



print("--- %s seconds ---" % (time.time() - start_time))

+-------+----+
|  price|view|
+-------+----+
|1250986|  89|
|1251616|  84|
|1251818|  94|
|1252469|  80|
|1253509|  85|
|1253637|  86|
|1254734|  85|
|1255252|  90|
|1255305|  98|
|1256433|  95|
|1256833|  82|
|1257157|  85|
|1258719|  88|
|1259627|  88|
|1259743|  77|
|1259916|  91|
|1259958|  89|
|1260462|  82|
|1260923|  75|
|1261005|  77|
+-------+----+
only showing top 20 rows

--- 0.5460398197174072 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")
